# Convert PDF to raw text

In [1]:
pwd

'd:\\Juan Tello\\ML\\OPEN AI\\openai-cookbook\\examples\\fine-tuned_qa'

In [2]:
file_path =r"examples\fine-tuned_qa\porce-III-data\Estudio amenaza sismica Porce III.pdf"

In [3]:
import PyPDF2

pdf_file = open(r'porce-III-data\Estudio amenaza sismica Porce III.pdf', 'rb')
read_pdf = PyPDF2.PdfReader(pdf_file)
number_of_pages = len(read_pdf.pages)
page = read_pdf.pages[7]
page_content = page.extract_text()

print(page_content)

EMPRESAS PÚBLICAS DE MEDELLÍN E.S.P.                                                1 
 
 
 
1. INTRODUCCIÓN 
 
 
Como parte de los estudios de complementación y actualización de la factibilidad del proyecto, la 
firma INGETEC S.A., realizó el estudio de evaluaci ón de la amenaza sísmica, el cual fue concluido 
y presentado en enero de 1998. El alcance previsto inicialmente para los estudios sismológicos de 
la etapa de diseño fue el de evaluar los parámetros sísmicos para el diseño sismorresistente de las 
obras del proyecto. El trabajo se inició en mayo  de 2002 con este único objetivo. Sin embargo, se 
consideró importante realizar antes una evaluaci ón de la nueva información sismológica disponible 
para la zona, representada en varios estudios rea lizados por diferentes firmas, durante el período 
comprendido entre enero/98 y mayo/02. La revisión indicó la necesidad de realizar una 
reevaluación de la amenaza sísmica del proyec to, ya que se disponía de nuevos datos y dado que 
lo

In [4]:
page_content

'EMPRESAS PÚBLICAS DE MEDELLÍN E.S.P.                                                1 \n \n \n \n1. INTRODUCCIÓN \n \n \nComo parte de los estudios de complementación y actualización de la factibilidad del proyecto, la \nfirma INGETEC S.A., realizó el estudio de evaluaci ón de la amenaza sísmica, el cual fue concluido \ny presentado en enero de 1998. El alcance previsto inicialmente para los estudios sismológicos de \nla etapa de diseño fue el de evaluar los parámetros sísmicos para el diseño sismorresistente de las \nobras del proyecto. El trabajo se inició en mayo  de 2002 con este único objetivo. Sin embargo, se \nconsideró importante realizar antes una evaluaci ón de la nueva información sismológica disponible \npara la zona, representada en varios estudios rea lizados por diferentes firmas, durante el período \ncomprendido entre enero/98 y mayo/02. La revisión indicó la necesidad de realizar una \nreevaluación de la amenaza sísmica del proyec to, ya que se disponía de nuevos dato

In [14]:
import re
from typing import Set
from transformers import GPT2TokenizerFast

import numpy as np
from nltk.tokenize import sent_tokenize

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    """count the number of tokens in a string"""
    return len(tokenizer.encode(text))

def reduce_long(
    long_text: str, long_text_tokens: bool = False, max_len: int = 590
) -> str:
    """
    Reduce a long text to a maximum of `max_len` tokens by potentially cutting at a sentence end
    """
    if not long_text_tokens:
        long_text_tokens = count_tokens(long_text)
    if long_text_tokens > max_len:
        sentences = sent_tokenize(long_text.replace("\n", " "))
        ntokens = 0
        for i, sentence in enumerate(sentences):
            ntokens += 1 + count_tokens(sentence)
            if ntokens > max_len:
                return ". ".join(sentences[:i][:-1]) + "."

    return long_text

discard_categories = ['See also', 'References', 'External links', 'Further reading', "Footnotes",
    "Bibliography", "Sources", "Citations", "Literature", "Footnotes", "Notes and references",
    "Photo gallery", "Works cited", "Photos", "Gallery", "Notes", "References and sources",
    "References and notes",]


def extract_sections(
    pdf_text: str,
    page_number: int,
    max_len: int = 1500,
    discard_categories: Set[str] = discard_categories,):
    """
    Extract the sections of a PDF document, discarding the references and other low information sections
    """
    # Split the text into lines
    lines = pdf_text.split('\n')
    sections = []
    current_section = {'heading': '', 'content': '', 'tokens': 0}
    for line in lines:
        # Check if the line matches the heading marker convention
        if re.match('^\d+\. .+', line):
            # If so, add the current section to the list and start a new section
            if current_section['tokens'] > 0:
                sections.append((
                    page_number,
                    current_section['heading'],
                    current_section['content'],
                    current_section['tokens']
                ))
            current_section = {'heading': line.strip(), 'content': '', 'tokens': 0}
        else:
            # Otherwise, add the line to the current section's content
            current_section['content'] += line + '\n'
            current_section['tokens'] += count_tokens(line) + 1  # Add 1 for the newline character
    # Add the last section to the list
    if current_section['tokens'] > 0:
        sections.append((
            page_number,
            current_section['heading'],
            current_section['content'],
            current_section['tokens']
        ))
    # Filter out low-information sections and truncate long sections
    outputs = []
    for section in sections:
        if section[3] < 40 or section[1] in discard_categories:
            continue
        elif section[3] > max_len:
            reduced_content = reduce_long(section[2], long_text_tokens=section[3], max_len=max_len)
            outputs.append((section[0], section[1], reduced_content, count_tokens(reduced_content)))
        else:
            outputs.append(section)
    return outputs



In [ ]:
### 1.2.1 We create a dataset and filter out any sections with fewer than 40 tokens, as those are unlikely to contain enough context to ask a good question.
res = []
for page_number, page in enumerate(read_pdf.pages):
    res += extract_sections(page.extract_text(), page_number)
    


In [20]:
import pandas as pd
df = pd.DataFrame(res, columns=["page_number", "heading", "content", "tokens"])
df = df[df.tokens>40]
# df = df.drop_duplicates(['heading'])
df = df.reset_index().drop('index',axis=1) # reset index
df.head(25)

,page_number,heading,content,tokens
0,0,,CONTRATO No. 030213329\nINFORME SISMOLÓGICO FI...,97
1,1,,EMPRESAS PÚBLICAS DE MEDELLÍN E.S.P. 1/4 \n \...,151
2,1,2. RESUMEN DE ESTUDIOS ANTERIORES ..............,2.1 GEOLOGÍA SÍSMICA Y SISMICIDAD DEL NOROESTE...,358
3,1,3. EVALUACIÓN DE AMEN AZA SÍSMICA ...............,3.1 METODOL OGÍA ................................,368
4,1,4. ESPECTROS Y SISMOS DE DISEÑO ................,4.1 METODOLOGIA .................................,132
5,1,5. CONCLUSIO NES ................................,INGETEC S.A. – KLO HN CRIPPEN CO . LTD. INFO ...,95
6,2,9. PROGRAMA Y REGISTRO DE R EVISIÓN Y VERIF IC...,\n \n \nINGETEC S.A. – KLO HN CRIPPEN CO . LT...,101
7,3,,EMPRESAS PÚBLICAS DE MEDELLÍN E.S.P. 3/4 \n \...,1189
8,4,,EMPRESAS PÚBLICAS DE MEDELLÍN E.S.P. 4/4 \nIN...,374
9,5,,EMPRESAS PÚBLICAS DE MEDELLÍN E.S.P. 1/2 \nLI...,743


In [21]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%